In [16]:
import os, glob, time
import numpy as np
from numpy.lib.stride_tricks import as_strided
from proc_utils import *

from gwdatafind import find_urls
#import gwpy as gwpy
from gwpy.io.gwf import get_channel_names
from gwpy.timeseries import TimeSeries
from gwpy.signal import filter_design
#import pycbc as pycbc
from pycbc import frame
from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector

import pyarrow.parquet as pq
import pyarrow as pa

import json

import matplotlib.pyplot as plt
# Not the same module the TimeSeries.Plot() calls, unfortunately
#plt.rcParams['pdf.fonttype'] = 42
#plt.rcParams['mathtext.fontset'] = 'cm'
%matplotlib inline

In [48]:
###################################################################
# Processing params
t_window  = 6 #16 #12 # s
t_stride  = 1 #4 #12 # s

In [47]:
###################################################################
# Read in inputs
#start = 1126256640 # GW150914 file start
#start = 1126259446 # GW150914: 32 s window start, until 1126259478

#start = 1126072320 # O1, first: lots of nans
start = 1126076416 # O1, second

#end_offset = 1
end_offset = t_window - t_stride # how much to read into following file, for continuity at file boundaries

end = 1126080512+end_offset # O1, third
#end = start+t_window #32 # s

end = 1137250304

print('>> Fetching data for GPS time: [%f, %f)'%(start, end))
print('.. total time: %f s'%(end-start))
ntimes = 1 + (end-start)//4096
print('.. N files expected:',ntimes)

#host = 'gw-openscience.org' # open data url but doesnt work, has to be done manually, but can also use pycbc get()
host = 'datafind.gw-openscience.org' # cvmfs, see: https://computing.docs.ligo.org/guide/cvmfs/#gwosc.osgstorage.org
#host = 'gw-openscience.org/archive/links/'
print('.. host:',host)
files = find_urls('H', 'H1_LOSC_4_V1', start, end, host=host) # fails if invalid range given (e.g. start > stop)

print(len(files))
'''
print('>> Input files...')
print('.. cwd:',os.getcwd())
# https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126072320-4096.gwf, 1st
# https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126076416-4096.gwf, 2nd
# ...
# https://www.gw-openscience.org/archive/data/O1/1136656384/H-H1_LOSC_4_V1-1137250304-4096.gwf, last
mydir = 'data/O1/test'
inputfiles = ['%s/%s'%(mydir, f.split('/')[-1]) for f in files]
assert len(inputfiles) == ntimes
for f in inputfiles:
    print('..',f)
    assert os.path.exists(f)
    
seed_from_file = int(inputfiles[0].split('-')[-2])
print('.. rand seed to be used:',seed_from_file)
'''

>> Fetching data for GPS time: [1126076416.000000, 1137250304.000000)
.. total time: 11173888.000000 s
.. N files expected: 2729
.. host: datafind.gw-openscience.org
1845


"\nprint('>> Input files...')\nprint('.. cwd:',os.getcwd())\n# https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126072320-4096.gwf, 1st\n# https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126076416-4096.gwf, 2nd\n# ...\n# https://www.gw-openscience.org/archive/data/O1/1136656384/H-H1_LOSC_4_V1-1137250304-4096.gwf, last\nmydir = 'data/O1/test'\ninputfiles = ['%s/%s'%(mydir, f.split('/')[-1]) for f in files]\nassert len(inputfiles) == ntimes\nfor f in inputfiles:\n    print('..',f)\n    assert os.path.exists(f)\n    \nseed_from_file = int(inputfiles[0].split('-')[-2])\nprint('.. rand seed to be used:',seed_from_file)\n"

In [23]:
print(os.getcwd())
f = open('data/O1/full/O1_list.json')
data = json.load(f)

/export/home/phys/mbandrews/work/gw


In [42]:
print(data.keys())
len(data['strain'][0])
print(data['strain'][0])
print(data['strain'][1])
print(type(data['strain'][1].keys()))
print(data['strain'][1]['format'])
gwf_data = []
for d in data['strain']:
    if d['format'] != 'gwf': continue
    gwf_data_ = {}
    for k in d.keys():
        gwf_data_[k] = d[k]
    gwf_data.append(gwf_data_)

dict_keys(['dataset', 'GPSstart', 'GPSend', 'strain'])
{'GPSstart': 1126072320, 'UTCstart': '2015-09-12T05:51:43', 'detector': 'H1', 'sampling_rate': 4096, 'duration': 4096, 'format': 'hdf5', 'url': 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126072320-4096.hdf5', 'min_strain': -3.93552169842707e-18, 'max_strain': 2.71796863909121e-18, 'mean_strain': -8.2569175207756e-25, 'stdev_strain': 1.86526167498977e-19, 'duty_cycle': 70.4833984375, 'BLRMS200': 1.14062458698724e-23, 'BLRMS1000': 5.27099187090827e-22, 'BNS': 51.8116043916147}
{'GPSstart': 1126072320, 'UTCstart': '2015-09-12T05:51:43', 'detector': 'H1', 'sampling_rate': 4096, 'duration': 4096, 'format': 'gwf', 'url': 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126072320-4096.gwf', 'min_strain': -3.93552169842707e-18, 'max_strain': 2.71796863909121e-18, 'mean_strain': -8.2569175207756e-25, 'stdev_strain': 1.86526167498977e-19, 'duty_cycle': 70.4833984375, 'BLRMS200': 1.140

In [52]:
len(gwf_data)
print(gwf_data[0]['url'])
test_url = gwf_data[0]['url']

https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126072320-4096.gwf


In [53]:
urls = [u['url'] for u in gwf_data]

In [56]:
len(urls)
print(urls[:10])
for u in urls[:10]:
    print(u)

['https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126072320-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126076416-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126080512-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126084608-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126088704-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126092800-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126096896-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126100992-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126105088-4096.gwf', 'https://www.gw-openscience.org/archive/data/O1/1125122048/H-H1_LOSC_4_V1-1126109184-4096.gwf']
https://www.gw-openscience.org/archive/

In [51]:
os.system('wget %s'%test_url)

0

In [8]:
len(files)
print(files[0])
times = [int(f.split('-')[-2]) for f in files]
print(times[0])

file://localhost/cvmfs/gwosc.osgstorage.org/gwdata/O1/strain.4k/frame.v1/H1/1125122048/H-H1_LOSC_4_V1-1126076416-4096.gwf
1126076416
